<a href="https://colab.research.google.com/github/juseonmin/bioinfo/blob/main/project-week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 생물정보학 및 실습 1 - 실습 프로젝트 : Figure 2E 구현하기

```
생물정보학 및 실습 1

2022-22456 주선민 

실습 프로젝트





## Google Drive 마운트

drive에 연결하기

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 실습용 Bioconda 환경 설치하기

In [3]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

Cloning into 'colab-biolab'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 69 (delta 22), reused 54 (delta 13), pack-reused 0
Unpacking objects: 100% (69/69), done.
./
./root/
./root/.vimrc
./root/.bashrc.biolab
./root/.bin.priority/
./root/.bin.priority/pip3
./root/.bin.priority/pip
./root/.bin.priority/pip2
./root/.tmux.conf
./root/.condarc
./root/.profile
--2022-05-27 12:45:40--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76607678 (73M) [application/x-sh]
Saving to: ‘miniconda3.sh’

miniconda3.sh       100%[===================>]  73.06M  19.1MB/s    in 4.0s    

2022-05-27 12:45:44 (18.2 MB/s) - ‘miniconda3.sh’ sav

## Term Project
논문의 Figure 2E를 재현하기 위한 코드 작성을 합니다. 먼저 binding site를 찾기 위해 read depth >= 50 으로 잘라줍니다. 
 (자외선에 의해서 단백질이 붙어있다가 단백질 분해효소로 단백질을 뜯어내고 남은 자리에서는 reversetranscriptase가 오류를 자주 냅니다. 오류율을 분석해서 단백질이 앉아 있었던 자리를 분석해 봅시다.)

In [5]:
%cd /content/drive/MyDrive/binfo1-work/

/content/drive/MyDrive/binfo1-work


mRNA motif를 조사하기 위해서 genome 전체 단위로 범위를 넓혀 pileup을 해줍니다. 

In [50]:
!samtools mpileup CLIP-35L33G.bam > CLIP-35L33G.pileup
!wc -l CLIP-35L33G.pileup

[mpileup] 1 samples in 1 input files
959697300 CLIP-35L33G.pileup


pileup 파일을 확인해줍니다.

In [4]:
!head CLIP-35L33G.pileup

head: cannot open 'CLIP-35L33G.pileup' for reading: No such file or directory


read depth가 50 이상인 것들로 추려냅니다.

In [52]:
!awk '$4 >= 50 { print $0; }' CLIP-35L33G.pileup > CLIP-35L33G-readcut.pileup
!head CLIP-35L33G-readcut.pileup

chr1	3222722	N	124	CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A	HCGGHEIHHHIBDDIIIGIG>GIHGDHHDDIAEIEEBIGIBGG8GIH@HEIHDIHDIIGHFFGFIIHI@HIGHIIIIIIIIGGBIBHHIHDGIIHBIF=GDDHIHGIHDHIHGBGBFGGHHGGF
chr1	3222723	N	146	GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G	E>HIHFIIHHIGDFIII<IHBIIHGDHEB?I@HIEIBIII;>G;GII@@HIHGIIEIIGIH0DIIIHIDHIBGIIIIIIIIIF?IBHHIHDGIIHGIBGEHDEI1HGHHEHFIGB@GH>GHDE>IGHIBHIEGGHGBGHGIGIDII
chr1	3222724	N	155	AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A^!A

이제 pandas에 불러와서 분석해 봅니다.

In [6]:
import pandas as pd

pileup = pd.read_csv('CLIP-35L33G-readcut.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
pileup.tail()

,chrom,pos,_ref,count,basereads,quals
25210334,MU069435.1,1564,N,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,DIDHGHIIGIGIEDIGIHIIG?FBGG@E=:A7G8@HHIHIIDDBII...
25210335,MU069435.1,1565,N,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,DIDHBHHIIIGICGIGIEIHDEFBGE8C=1?1G88IHIHIIDGBHI...
25210336,MU069435.1,1566,N,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,DIDHG>IIHIHHCEIIIEB5D@;=H5BIHIHHI<DAIIBDIIFIIH...
25210337,MU069435.1,1567,N,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,DIDHFHGIIIIIDDIII?B=G::B=G=DIHIHIID@BIIGGIIHIH...
25210338,MU069435.1,1568,N,88,>>>>T$T$T$T$T$T$T$T$T$T$TT$T$TT$T$T$T$T$T$T$T$...,DIDHAG4BDBHBEIGHIHHHIIHIDIIIIIHIGIHIIEIHIHDHIG...


match와 substitution에 해당하는 tag 이외에 제거하기

In [ ]:
import re
toremove = re.compile('[<>$*#]')
pileup['matches'] = pileup['basereads'].apply(lambda x: toremove.sub('', x))

In [ ]:
df_length = pileup[['chrom', 'pos', 'matches']]

[Shannon entropy 구해서 cut 하기]
1. 각 position별로 base수를 셉니다.
1. 각 position별로 Shannon entropy를 계산합니다.
1.  bedgraph format(https://genome.ucsc.edu/goldenPath/help/bedgraph.html)으로 출력.( chr/start pos/end pos/entropy )

1. 각 position 별로 base 수 세기
#### - ^ (caret) marks the start of a read segment and the ASCII of the character following `^' minus 33 gives the mapping quality 
#### -즉 ^G는 sequence가 아닌 mapping qualit를 나타내기 때문에 G로 세는 것이 아니라 빼주어야 함


In [ ]:
for i in 'A','T','C':
  df_length[i] = df_length['matches'].str.count(i)
df_length['G'] = df_length['matches'].str.count('G') - df_length['matches'].str.count("\^G")
df_length

In [ ]:
df_length['length'] = df_length[['A','T','C','G']].sum(axis=1)
df_length

### Crosslinking-induced reverse-transcription error score (CRES) 즉, Shannon Entropy 구하기

In [ ]:
new_table = pd.DataFrame()
for i in 'A','T','C','G':
    new_table[i] = np.log2(1/(df_length[i]/df_length['length']))*(df_length[i]/df_length['length'])
new_table



In [ ]:
df_length['entropy'] = new_table.sum(axis=1)
df_length 

In [ ]:
df_length['pos_end'] = df_length['pos'].apply(lambda x : x+1) 
df_cut_entropy = df_length.loc[df_length['entropy'] >= 0.8]
df_cut_entropy


In [ ]:
cutdata = df_cut_entropy[['chrom', 'pos','pos_end','entropy']]
cutdata.to_csv('cutdata.txt', sep = ' ', index=None)